In [1]:
import json
import csv
import re
from datetime import date,datetime,timedelta
from pathlib import Path
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import yfinance as yf

In [2]:
def stock_data(ticker):
    start_date = '2022-03-01'
    end_date = date.today()+timedelta(days=1)
    # 1. Request data:
    data = yf.download(ticker, 
                      start=start_date, 
                      end=end_date,
                      interval='1d', 
                      progress=False)
    # 2. Feature Engineering:
    data['Percent Price Change Within Period'] = ((data['Close'] - data['Open'])/data['Open'])*100
    # data['Change in Close Price'] = data['Close'] - data['Close'].shift(1)
    # data['Scaled Delta Close'] = data['Change in Close Price']/(data['Close'].mean())
    data['Scaled Volume'] = data['Volume']/data['Volume'].mean()
    data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
    data['SMA(3)'] = data_SMA
    data['t+1'] = data['Adj Close'].shift(-1)
    data.reset_index(inplace=True)
    #data['Datetime']=data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
    data.drop(['Close'],axis=1,inplace=True)
    #3. Export data:
    #f_name = ticker + "_data"
    # data.to_csv('~/LighthouseLabs-Final/Dataset/1. Stock_Data/' + f_name + ".csv")
    # print('Data saved!')
    return data
    

In [3]:
technical_analysis_df = pd.DataFrame(stock_data('TSLA'))
technical_analysis_df

,Date,Open,High,Low,Adj Close,Volume,Percent Price Change Within Period,Scaled Volume,SMA(3),t+1
0,2022-03-01,869.679993,889.880005,853.780029,864.369995,24922300,-0.610569,1.006451,NaN,879.890015
1,2022-03-02,872.130005,886.479980,844.270020,879.890015,24881100,0.889777,1.004787,NaN,839.289978
2,2022-03-03,878.770020,886.440002,832.599976,839.289978,20541200,-4.492648,0.829527,NaN,838.289978
3,2022-03-04,849.099976,855.650024,825.159973,838.289978,22333200,-1.273112,0.901894,861.183329,804.580017
4,2022-03-07,856.299988,866.140015,804.570007,804.580017,24164700,-6.039936,0.975856,852.489990,824.400024
5,2022-03-08,795.530029,849.989990,782.169983,824.400024,26799700,3.629026,1.082267,827.386658,858.969971
6,2022-03-09,839.479980,860.559998,832.010010,858.969971,19728000,2.321674,0.796687,822.423340,838.299988
7,2022-03-10,851.450012,854.450012,810.359985,838.299988,19549500,-1.544427,0.789478,829.316671,795.349976
8,2022-03-11,840.200012,843.799988,793.770020,795.349976,22272800,-5.338019,0.899455,840.556661,766.369995
9,2022-03-14,780.609985,800.700012,756.039978,766.369995,23717400,-1.824213,0.957793,830.873311,801.890015


In [4]:
technical_analysis_df.set_index('Date', inplace = True)

In [5]:
technical_analysis_df

,Open,High,Low,Adj Close,Volume,Percent Price Change Within Period,Scaled Volume,SMA(3),t+1
Date,,,,,,,,,
2022-03-01,869.679993,889.880005,853.780029,864.369995,24922300,-0.610569,1.006451,NaN,879.890015
2022-03-02,872.130005,886.479980,844.270020,879.890015,24881100,0.889777,1.004787,NaN,839.289978
2022-03-03,878.770020,886.440002,832.599976,839.289978,20541200,-4.492648,0.829527,NaN,838.289978
2022-03-04,849.099976,855.650024,825.159973,838.289978,22333200,-1.273112,0.901894,861.183329,804.580017
2022-03-07,856.299988,866.140015,804.570007,804.580017,24164700,-6.039936,0.975856,852.489990,824.400024
2022-03-08,795.530029,849.989990,782.169983,824.400024,26799700,3.629026,1.082267,827.386658,858.969971
2022-03-09,839.479980,860.559998,832.010010,858.969971,19728000,2.321674,0.796687,822.423340,838.299988
2022-03-10,851.450012,854.450012,810.359985,838.299988,19549500,-1.544427,0.789478,829.316671,795.349976
2022-03-11,840.200012,843.799988,793.770020,795.349976,22272800,-5.338019,0.899455,840.556661,766.369995


In [6]:
csvpath = Path("./technical_analysis_df.csv")
technical_analysis_df.to_csv(csvpath)